In [4]:
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
import re
import pandas as pd
from collections import defaultdict

# --- 1. 定义文件路径 ---
excel_file = "共线表_41588_2023_1620_MOESM2_ESM.xlsx"
sheet_name = "Supplementary Table 2a"
human_fasta_file = "gencode.v25.lncRNA_transcripts.fa"
mouse_fasta_file = "gencode.vM10.lncRNA_transcripts.fa"
output_human_fasta = "human_syntenic_lncRNAs.fa"
output_mouse_fasta = "mouse_syntenic_lncRNAs.fa"

# --- 2. 读取Excel共线对文件 ---
print("正在读取Excel文件...")
try:
    synteny_df = pd.read_excel(excel_file, sheet_name=sheet_name)
    
    print("DataFrame列名:", synteny_df.columns.tolist())
    print("前5行数据:")
    print(synteny_df.head())
    
    human_col = synteny_df.columns[0]  # 第一列
    mouse_col = synteny_df.columns[1]  # 第二列
    
    print(f"使用列: 人类基因 - '{human_col}', 小鼠基因 - '{mouse_col}'")
    
    # 提取基因ID对
    synteny_pairs = []
    human_gene_ids = set()
    mouse_gene_ids = set()
    
    for index, row in synteny_df.iterrows():
        human_id = str(row[human_col]).strip()
        mouse_id = str(row[mouse_col]).strip()
        
        # 跳过空值或无效值
        if human_id != 'nan' and mouse_id != 'nan' and human_id and mouse_id:
            # 去除版本号（保留基因ID部分）
            if '.' in human_id:
                human_id = human_id.split('.')[0]
            if '.' in mouse_id:
                mouse_id = mouse_id.split('.')[0]
            
            synteny_pairs.append((human_id, mouse_id))
            human_gene_ids.add(human_id)
            mouse_gene_ids.add(mouse_id)
    
    print(f"共读取 {len(synteny_pairs)} 对有效共线基因对。")
    print(f"包含 {len(human_gene_ids)} 个人类基因和 {len(mouse_gene_ids)} 个小鼠基因。")
    
except Exception as e:
    print(f"读取Excel文件时出错: {e}")
    exit()

# --- 3. 构建基因ID到最长转录本ID的映射 ---
def build_gene_to_longest_transcript(fasta_file, species_genes):
    """
    为指定物种构建基因ID到最长转录本ID的映射
    """
    gene_to_transcript = {}
    transcript_lengths = defaultdict(list)
    
    print(f"正在分析 {fasta_file} 中的转录本...")
    
    # 第一次遍历：记录所有转录本的长度
    for record in SeqIO.parse(fasta_file, "fasta"):
        header = record.id
        seq_length = len(record.seq)
        
        # 从header中提取基因ID（去除版本号）
        # GENCODE格式: ENSTXXXXX.X|ENSGXXXXX.X|...
        parts = header.split('|')
        if len(parts) >= 2:
            gene_id_with_version = parts[1]
            gene_id = gene_id_with_version.split('.')[0]  # 去除版本号
            
            # 只关注我们需要的基因
            if gene_id in species_genes:
                transcript_lengths[gene_id].append((header, seq_length))
    
    # 第二次遍历：为每个基因选择最长的转录本
    for gene_id, transcripts in transcript_lengths.items():
        if transcripts:
            # 按序列长度排序，选择最长的
            transcripts.sort(key=lambda x: x[1], reverse=True)
            longest_transcript = transcripts[0][0]  # 最长的转录本header
            gene_to_transcript[gene_id] = longest_transcript
    
    print(f"为 {len(gene_to_transcript)}/{len(species_genes)} 个基因找到了转录本")
    return gene_to_transcript

# 为人类和小鼠分别构建映射
human_gene_to_transcript = build_gene_to_longest_transcript(human_fasta_file, human_gene_ids)
mouse_gene_to_transcript = build_gene_to_longest_transcript(mouse_fasta_file, mouse_gene_ids)

# --- 4. 创建转录本ID的集合用于快速查找 ---
human_transcript_ids = set(human_gene_to_transcript.values())
mouse_transcript_ids = set(mouse_gene_to_transcript.values())

print(f"人类最长转录本数量: {len(human_transcript_ids)}")
print(f"小鼠最长转录本数量: {len(mouse_transcript_ids)}")

# --- 5. 索引FASTA文件 ---
print("正在索引人类FASTA文件...")
human_idx = SeqIO.index(human_fasta_file, "fasta")
print("正在索引小鼠FASTA文件...")
mouse_idx = SeqIO.index(mouse_fasta_file, "fasta")

# --- 6. 提取序列并写入新文件 ---
def write_single_line_fasta(record, output_handle):
    """将序列写入为单行FASTA格式"""
    output_handle.write(f">{record.id}\n")
    output_handle.write(f"{str(record.seq)}\n")

with open(output_human_fasta, 'w') as human_out, open(output_mouse_fasta, 'w') as mouse_out:
    human_count = 0
    mouse_count = 0
    processed_pairs = 0
    found_pairs = 0

    # 遍历每一对共线基因
    for human_gene_id, mouse_gene_id in synteny_pairs:
        found_human = False
        found_mouse = False
        
        # 获取人类基因的最长转录本ID
        if human_gene_id in human_gene_to_transcript:
            transcript_id = human_gene_to_transcript[human_gene_id]
            if transcript_id in human_idx:
                seq_record = human_idx[transcript_id]
                write_single_line_fasta(seq_record, human_out)
                found_human = True
                human_count += 1
        
        # 获取小鼠基因的最长转录本ID
        if mouse_gene_id in mouse_gene_to_transcript:
            transcript_id = mouse_gene_to_transcript[mouse_gene_id]
            if transcript_id in mouse_idx:
                seq_record = mouse_idx[transcript_id]
                write_single_line_fasta(seq_record, mouse_out)
                found_mouse = True
                mouse_count += 1
        
        # 统计找到的配对
        if found_human and found_mouse:
            found_pairs += 1
        
        processed_pairs += 1
        if processed_pairs % 1000 == 0:
            print(f"已处理 {processed_pairs}/{len(synteny_pairs)} 对共线基因...")
            print(f"  找到 {found_pairs} 对完全匹配的基因")

print("\n完成！")
print(f"从人类文件中提取了 {human_count} 条序列到 {output_human_fasta}")
print(f"从小鼠文件中提取了 {mouse_count} 条序列到 {output_mouse_fasta}")
print(f"成功找到 {found_pairs}/{len(synteny_pairs)} 对完全匹配的基因")

# 关闭索引以释放资源
human_idx.close()
mouse_idx.close()

# 显示一些统计信息
print("\n=== 统计信息 ===")
print(f"总共线对: {len(synteny_pairs)}")
print(f"找到人类转录本的基因: {len(human_gene_to_transcript)}")
print(f"找到小鼠转录本的基因: {len(mouse_gene_to_transcript)}")
print(f"成功提取的人类序列: {human_count}")
print(f"成功提取的小鼠序列: {mouse_count}")

# # 可选：保存基因到转录本的映射关系
# if human_gene_to_transcript:
#     with open("human_gene_to_transcript_mapping.txt", 'w') as f:
#         for gene_id, transcript_id in human_gene_to_transcript.items():
#             f.write(f"{gene_id}\t{transcript_id}\n")

# if mouse_gene_to_transcript:
#     with open("mouse_gene_to_transcript_mapping.txt", 'w') as f:
#         for gene_id, transcript_id in mouse_gene_to_transcript.items():
#             f.write(f"{gene_id}\t{transcript_id}\n")

正在读取Excel文件...
DataFrame列名: ['human gene', 'mouse gene']
前5行数据:
        human gene          mouse gene
0  ENSG00000182873  ENSMUSG00000085063
1  ENSG00000182873  ENSMUSG00000098729
2  ENSG00000215014  ENSMUSG00000073679
3  ENSG00000215014  ENSMUSG00000086549
4  ENSG00000215014  ENSMUSG00000086682
使用列: 人类基因 - 'human gene', 小鼠基因 - 'mouse gene'
共读取 25005 对有效共线基因对。
包含 9721 个人类基因和 7565 个小鼠基因。
正在分析 gencode.v25.lncRNA_transcripts.fa 中的转录本...
为 9721/9721 个基因找到了转录本
正在分析 gencode.vM10.lncRNA_transcripts.fa 中的转录本...
为 7565/7565 个基因找到了转录本
人类最长转录本数量: 9721
小鼠最长转录本数量: 7565
正在索引人类FASTA文件...
正在索引小鼠FASTA文件...
已处理 1000/25005 对共线基因...
  找到 1000 对完全匹配的基因
已处理 2000/25005 对共线基因...
  找到 2000 对完全匹配的基因
已处理 3000/25005 对共线基因...
  找到 3000 对完全匹配的基因
已处理 4000/25005 对共线基因...
  找到 4000 对完全匹配的基因
已处理 5000/25005 对共线基因...
  找到 5000 对完全匹配的基因
已处理 6000/25005 对共线基因...
  找到 6000 对完全匹配的基因
已处理 7000/25005 对共线基因...
  找到 7000 对完全匹配的基因
已处理 8000/25005 对共线基因...
  找到 8000 对完全匹配的基因
已处理 9000/25005 对共线基因...
  找到 9000 对完全匹配的基因
已处理 10000/25005